In [ ]:
from main import main

main()

### hadeles cookes categoria predition

In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import pandas as pd
import re
engine = create_engine("sqlite:///produtos.db")
Session = sessionmaker(bind=engine)
session = Session()

query = """
SELECT link_imagem 
FROM imagens;
"""
df = pd.read_sql_query(query, engine)


# Função para extrair sufixo e extensão
def extrair_sufixo_extensao(url):
    match = re.search(r'/(\d+)(_.*?)(\.\w+)(\?|$)', url)
    if match:
        return match.group(2), match.group(3)  # Retorna o sufixo e a extensão
    return None, None

# Aplicar a função e expandir para colunas separadas
df[['Sufixo', 'Extensao']] = df['link_imagem'].apply(lambda x: pd.Series(extrair_sufixo_extensao(x)))
#display(df[['Sufixo', 'Extensao']].value_counts())
#display(df[['Sufixo']].value_counts())
#display(df[[ 'Extensao']].value_counts())

In [1]:
import aiohttp
import asyncio
import time
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy.sql import text
import re

# Cabeçalhos HTTP para as requisições
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
MODIFICADORES = ["_1", "_99_1", "_1_1", "_1_3", "_99_3"]
EXTENSOES = [".jpg", ".png"]

engine = create_engine("sqlite:///produtos.db")
Session = sessionmaker(bind=engine)
session = Session()
query = """
SELECT id, link 
FROM produtos
WHERE id NOT IN (SELECT produto_id FROM imagens);
"""

In [ ]:

def extrair_numero(link):
    """Extrai o ID único do link."""
    match = re.search(r'-(\d+)$', link)
    if match:
        return match.group(1)
    return None

def salvar_imagem_no_banco(id, url_imagem, session):
    """
    Atualiza o campo 'conteudo' na tabela 'imagens' com os dados binários da imagem.
    :param url: URL da imagem (string)
    :param imagem_binaria: Conteúdo da imagem em bytes
    :param session: Sessão do SQLAlchemy
    """
    try:
        # Query de atualização usando text
        query = text("INSERT INTO imagens (link_imagem, produto_id) VALUES (:url, :id)")
        session.execute(query, {"url": url_imagem, "id": id})
        session.commit()
        print(f"Imagem salva no banco com sucesso para o URL: {url_imagem}")
    except Exception as e:
        session.rollback()
        print(f"Erro ao salvar a imagem no banco para o URL {url_imagem}: {e}")

# Função para fazer requisições HTTP e salvar no banco
async def make_request(url, session, db_session, id):
    """
    Envia uma requisição HTTP assíncrona e salva a imagem no banco de dados.
    :param url: URL para fazer a requisição
    :param session: Sessão HTTP assíncrona
    :param db_session: Sessão do banco de dados
    """
    try:
        async with session.head(url, headers=HEADERS) as response:
            status = response.status            
            if status == 200:
                # Salvar a imagem no banco
                salvar_imagem_no_banco(id, url, db_session)
            return status
    except Exception as e:
        print(f"Erro na requisição : {e}")
        return None

# Função principal para processar requisições
async def main(total_requests, db_session, recovery_time=10, ):
    """
    Processa requisições assíncronas e salva imagens no banco de dados.
    :param total_requests: Lista de URLs para processar
    :param db_session: Sessão do banco de dados
    :param recovery_time: Tempo de espera em caso de status 202
    """
    start_time = time.time()
    completed_requests = 0  # Contador de requisições concluídas

    async with aiohttp.ClientSession() as session:
        for row in total_requests.itertuples(index=False):
            id, link = row
            codigo = extrair_numero(link)
            
            base_url = f"https://conteudo.irmaosgoncalves.com.br/produto/{codigo}/{codigo}"
            for modificador in MODIFICADORES:
                if encontrou_valido:
                    break  
                for extensao in EXTENSOES:
                    url = f"{base_url}{modificador}{extensao}?width=400"
                    print(f"Iniciando requisição {completed_requests + 1} de {len(total_requests)}")
                    status = await make_request(url, session, db_session, id)

                    
                    while status == 202:
                        print(f"Status 202 recebido na requisição {completed_requests + 1}. Aguardando {recovery_time} segundos...")
                        await asyncio.sleep(recovery_time)  
                        status = await make_request(url, session, db_session, id)  

                    if status == 404:
                        continue

                    elif status == 200:  
                        encontrou_valido = True  
                        completed_requests += 1
                        break

                    completed_requests += 1


    elapsed_time = time.time() - start_time
    print(f"Todas as requisições concluídas. Tempo total: {elapsed_time:.2f} segundos")
    

def baixar_imagem():
    df = pd.read_sql_query(query, engine)
    # Executar as requisições e salvar imagens no banco
    asyncio.run(main(total_requests=df, db_session=session))

In [2]:
df = pd.read_sql_query(query, engine)
display(df)
def extrair_numero(link):
    """Extrai o ID único do link."""
    match = re.search(r'-(\d+)$', link)
    if match:
        return match.group(1)
    return None

for row in df.itertuples(index=False):
    a, b = row
    print(extrair_numero(b)) 

,id,link
0,10035,https://www.irmaosgoncalves.com.br/produto/bar...
1,10036,https://www.irmaosgoncalves.com.br/produto/bar...
2,10037,https://www.irmaosgoncalves.com.br/produto/bar...
3,10038,https://www.irmaosgoncalves.com.br/produto/bar...
4,10039,https://www.irmaosgoncalves.com.br/produto/bar...
...,...,...
16435,5830,https://www.irmaosgoncalves.com.br/produto/óle...
16436,13069,https://www.irmaosgoncalves.com.br/produto/óle...
16437,5831,https://www.irmaosgoncalves.com.br/produto/óle...
16438,5832,https://www.irmaosgoncalves.com.br/produto/ólo...


609963966264
7898755180428
7898755180343
7896311764785
7896311773275
7896311773299
7896311768332
7896311710331
7896311764808
7898755180107
7898755180596
7898115901854
7898937674356
602883464506
7899706149570
7899706149600
7899706149594
7899706179560
7899895724862
7896132133135
7896132133159
7891114152692
7891114152685
7896082500513
7896056406254
602883084117
02349
7896221401640
7896221402487
7896221400599
7892840823375
7891095023172
7892840818142
7892840822958
7892840823030
7892840823047
7892840818210
7892840817978
7892840253646
7892840823054
7892840818203
7891095023141
7891095023110
7891095023127
06613
7896221400032
7896221402593
7891962066899
7891962066882
7891962066875
7891095031122
7891095031153
7891095031160
7891095911592
7891095031177
7891095031115
7891095031139
7896183907334
7896183907327
7894904574100
7896004008998
7896004006406
7896004010137
7896004006239
7896004010465
38000304057
7896004006277
7896004010076
7896004006284
7898921567404
7898921567275
7898921567503
7898921567497

In [ ]:
import pandas as pd
import numpy as np
import time

# Criar um DataFrame com 18.000 linhas
data = {'coluna1': np.random.randint(1, 100, 18000),
        'coluna2': np.random.randint(1, 100, 18000)}
df = pd.DataFrame(data)

# Usando .iterrows()
start = time.time()
for _, row in df.iterrows():
    _ = row['coluna1'], row['coluna2']
print(f"Iterrows: {time.time() - start:.5f} segundos")

# Usando .itertuples()
start = time.time()
for row in df.itertuples(index=False):
    _ = row[0], row[1]
print(f"Itetuples: {time.time() - start:.5f} segundos")

# Usando .to_numpy()
start = time.time()
for value1, value2 in df.to_numpy():
    _ = value1, value2
print(f"To Numpy: {time.time() - start:.5f} segundos")
